In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
print(os.environ.get('GPU')) # Empty # This is Correct

None


In [5]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 32.1 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 114.7 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 104.3 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 3.6 MB/s s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=bf666d7cdd7498f075ccba2f968bc4bce17396553e13b59e9ce02850919ca269
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0133f0ac8d8b628755da8c3b0ce3c31b261bd7841db123cdfdaa48a2ca1b763f
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built promise pathtools


In [6]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

 ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
#721fdeb49f2aeea9c5eddef6eea98f3a1ce586d0

In [8]:
wandb.init(project="w251_hw9", entity="seankoval")

wandb: Currently logged in as: seankoval. Use `wandb login --relogin` to force relogin


In [9]:
SEED=1

random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [10]:
torch.cuda.device_count()

1

In [11]:
START_EPOCH = 0

In [12]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
TRAINDIR="/data/train"
VALDIR="/data/val"

In [13]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [14]:
#wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [15]:
GPU = 0

In [16]:
torch.cuda.set_device(GPU)

In [17]:
global_step = 0

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [19]:
def validate(val_loader, model, criterion):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    global_step = global_step + 1

    return top1.avg

In [20]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [21]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [22]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [23]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [24]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [25]:
cudnn.benchmark = True

In [26]:
import torch.distributed as dist
import sys

In [27]:
WORLD_SIZE = 2
BACKEND = 'nccl'
# tcp://172.31.17.123:443
URL = 'tcp://54.173.7.235:1883'

RANK = 0

dist.init_process_group(backend=BACKEND, init_method=URL, rank=RANK, world_size=WORLD_SIZE)

In [28]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [29]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [30]:
IMG_SIZE = 224
NUM_CLASSES = 1000

In [31]:
model = models.__dict__[ARCH]()

In [32]:
inf = model.fc.in_features

In [33]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [34]:
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [35]:
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

ip-172-31-26-243:412:412 [0] NCCL INFO Bootstrap : Using ens5:172.31.26.243<0>
ip-172-31-26-243:412:412 [0] NCCL INFO Plugin Path : /opt/hpcx/nccl_rdma_sharp_plugin/lib/libnccl-net.so
ip-172-31-26-243:412:412 [0] NCCL INFO P2P plugin IBext
ip-172-31-26-243:412:412 [0] NCCL INFO NET/IB : No device found.
ip-172-31-26-243:412:412 [0] NCCL INFO NET/IB : No device found.
ip-172-31-26-243:412:412 [0] NCCL INFO NET/Socket : Using [0]ens5:172.31.26.243<0>
ip-172-31-26-243:412:412 [0] NCCL INFO Using network Socket
NCCL version 2.10.3+cuda11.4
ip-172-31-26-243:412:524 [0] NCCL INFO Channel 00/02 :    0   1
ip-172-31-26-243:412:524 [0] NCCL INFO Channel 01/02 :    0   1
ip-172-31-26-243:412:524 [0] NCCL INFO Trees [0] 1/-1/-1->0->-1 [1] -1/-1/-1->0->1
ip-172-31-26-243:412:524 [0] NCCL INFO Channel 00 : 1[1e0] -> 0[1e0] [receive] via NET/Socket/0
ip-172-31-26-243:412:524 [0] NCCL INFO NET/Socket: Using 2 threads and 8 sockets per thread
ip-172-31-26-243:412:524 [0] NCCL INFO Channel 01 : 1[1e0] 

In [36]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [37]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [38]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [39]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [40]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [41]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [42]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [43]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [44]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [45]:
best_acc1 = 0

In [46]:
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()[0]))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 15.057 (15.057)	Data  3.466 ( 3.466)	Loss 7.0052e+00 (7.0052e+00)	Acc@1   0.00 (  0.00)	Acc@5   1.00 (  1.00)
Epoch: [0][  10/1282]	Time  2.463 ( 2.849)	Data  0.003 ( 0.450)	Loss 6.9297e+00 (6.9718e+00)	Acc@1   0.40 (  0.11)	Acc@5   1.00 (  0.65)
Epoch: [0][  20/1282]	Time  2.335 ( 2.367)	Data  0.003 ( 0.251)	Loss 6.9399e+00 (6.9599e+00)	Acc@1   0.20 (  0.14)	Acc@5   1.80 (  0.90)
Epoch: [0][  30/1282]	Time  3.857 ( 2.250)	Data  0.003 ( 0.183)	Loss 6.8102e+00 (6.9268e+00)	Acc@1   0.40 (  0.23)	Acc@5   1.00 (  1.12)
Epoch: [0][  40/1282]	Time  1.983 ( 2.188)	Data  0.003 ( 0.139)	Loss 6.7301e+00 (6.8912e+00)	Acc@1   1.00 (  0.33)	Acc@5   3.60 (  1.39)
Epoch: [0][  50/1282]	Time  2.560 ( 2.197)	Data  0.002 ( 0.118)	Loss 6.7253e+00 (6.8476e+00)	Acc@1   0.20 (  0.37)	Acc@5   1.80 (  1.63)
Epoch: [0][  60/1282]	Time  2.810 ( 2.133)	Data  0.002 ( 0.099)	Loss 6.5416e+00 (6.8051e+00)	Acc@1   1.60 (  0.47)	Acc@5   3.40 (  1.88)
Epoch: [0][  70/1282]	Time  3.270 ( 2.089

In [47]:
writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs